In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas as pd

# all units SI (meters, kg, etc.)
distance = 7
height = 2.495
m = 0.141748
R_l = 0.0508
R_u = 0.03175
R_b = 0.0889
A = 0.0248286665
interval = 0.001
run = True

## Create Functions

In [2]:
def iterate():
    global run
    for w_r in range(30,600): #6380 falcon 500 rpm in radians/s
        for theta in range(15,85):
            if(run):
                trajectory(w_r, theta*math.pi/180)
    #print("done")
    run = True

In [3]:
def trajectory(w, theta):
    v = (w*R_l + 9/16*w*R_u)/2
    #print(v)
    s = (w*R_l - 9/16*w*R_u)/(2*math.pi*R_b)
    runPath(w,v,s,theta)

In [7]:
def runPath(w,v_i,s,theta_i):
    v_x = v_i*math.cos(theta_i)
    v_y = v_i*math.sin(theta_i)
    theta = theta_i
    x_dist = 0
    y_dist = 0
    through = True
    x = [0]
    y = [0]
    while(through):
        x_dist += v_x*interval
        y_dist += v_y*interval
        x.append(x_dist)
        y.append(y_dist)
        v_n = math.sqrt(v_x*v_x + v_y*v_y)
        theta = angle(v_x,v_y)
     
        magnus = magnusF(s, v_n)
        drag = dragF(v_n)
        v_x -= (magnus*math.sin(theta) + drag*math.cos(theta))*interval
        v_y -= (drag*math.sin(theta) + 9.8 - magnus*math.cos(theta))*interval
        
        if(v_y < 0.1 and v_y > -0.1):
            if(((x_dist > distance - .01) and (x_dist < distance + .01)) 
            and ((y_dist > height - .015) and (y_dist < height + .015))):
                through = False
                global w_min
                plt.plot(x,y)
                omega_array.append(round(w*60/(2*math.pi),1))
                theta_array.append(round(theta_i*180/math.pi,1))
                print ("w: " + str(w*60/(2*math.pi)) + " a: " + str(theta_i*180/math.pi))
                global run
                run = False
  
        if(v_y < -0.15):
            through = False
      

In [8]:
def magnusF(s,v):
    return (0.1)*(16/3)*(math.pi*math.pi)*1.225*(R_b**3)*s*v/m
def dragF(v):
    return (0.47)*(1.225/2)*A*(v**2)/m
def angle(x,y):
    return math.atan(y/x)

In [ ]:
plt.title("Peak-Based Ball Trajectory at Varying Distances, \n$C_l$ = 0.1, $C_f$ = 0.47")
plt.xlabel("Distance (m)")
plt.ylabel("Height (m)")

plt.rcParams.update({'mathtext.default':  'regular' })

trial_array = []
distance_array = []
omega_array = []
theta_array = []
max_trials = 50

for i in range(0,max_trials):
    trial_array.append(i+1)
    distance = 2.3 + ((7-2.3) / max_trials)*i
    distance_array.append(distance)
    print(str(i+1) + " , " + str(distance))
    iterate()

plt.savefig("peak_based_" + str(max_trials) + "_trials.png")
plt.show()

1 , 2.3


In [104]:
print(len(trial_array))
print(len(distance_array))
print(len(omega_array))
print(len(theta_array))

50
50
48
48


In [105]:
# print(trial_array.pop(9), distance_array.pop(9))
# # trial_array[6]

## Test

In [2]:
def make_bold(val):
    bold = 'bold'
    
    return 'font-weight: %s' % bold

df = pd.DataFrame(data = {'Trial': trial_array, 
                          'Distance': distance_array, 
                          'Angular Velocity': omega_array, 
                          'Angle': theta_array})
print(df)
df.to_csv("peak_based_" + str(max_trials) + "_trials.csv")

NameError: name 'pd' is not defined